# Настройка ноутбука

In [19]:
import pandas as pd
import numpy as np
import datetime as dt

from pandasql import sqldf

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Расширить рабочее поле ноутбука на весь экран
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Описание

В этом ноутбуки я документирую примеры update-запросов, а также описание ситуаций их применения. В моей практике, update запросы обычно выполняются при проведении миграций в базу. 

# Кейсы

### Упаковка значений колонок в json-формат

При разработке конвейера обработки данных, возникла потребность изменить тип колонки в базе данных со строкового на json.
Для этого нужно было:

1. Упаковать все непустые значения в колонке (current_col) в установленный json-формат ({'json_key': col_value})
2. Заменить все пустые строки на обычные null значения.
3. Поменять тип колонки на json (точнее jsonb)

Ниже приведена последовательность запросов, которые выполняют эти шаги:

```sql
WITH json_vals AS (
    SELECT
          json_build_object('json_key', json_agg(current_col)) AS packed_value
        , chat_id, mes_num
    FROM 
        scheme.tab
    WHERE 
        current_col IS NOT NULL AND button_id <> ''
    GROUP BY chat_id, mes_num
)
UPDATE scheme.tab final_tab SET current_col = json_vals.packed_value FROM json_vals
WHERE 
    final_tab.chat_id = json_vals.chat_id 
    AND
    final_tab.mes_num = json_vals.mes_num;

UPDATE scheme.tab SET current_col = nullif(current_col, '')::jsonb;
ALTER tab scheme.tab ALTER COLUMN current_col TYPE jsonb USING current_col::jsonb;
```

### Заменить конкретные значения колонки на строковый 'null'

Иногда возникает необходимость удалить конкретные значения колонки, при этом не удоляя всю строку, тоесть заменить его пустым значением. В моем случае, из за неполадки в одной из функций, на месте пустых значений выгружались строки 'null'. Ниже приведен запрос, который заменяет их на обычные null-значения:

```sql
UPDATE scheme.tab SET current_table = NULL WHERE current_table = 'null'
```

### Добавить новую колонку

```sql
ALTER TABLE scheme.tab ADD COLUMN new_col integer;
```

### Переименовать колонку

```sql
ALTER TABLE scheme.tab RENAME COLUMN old_col_name TO new_col_name;
```

### Удалить колонку

```sql
ALTER TABLE scheme.tab DROP COLUMN col_to_delete;
```